In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
print("cwd ", os.getcwd())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cs522-paper/Runs.csv
/kaggle/input/cs522-paper/Counts.csv
/kaggle/input/cs522-paper/Stops.csv
/kaggle/input/cs522-paper/Coordinates.csv
cwd  /kaggle/working


In [2]:
# find the average of a list of numbers
def avgList(lst):
    acc = 0
    for elt in lst :
        acc += elt
    return acc / len(lst)
    
# determine coordinates for each route unit
csvpd = pd.read_csv('../input/cs522-paper/Coordinates.csv')
coordDict = {}
RTNUM = 'RouteNumber'
LAT = 'lat'
LON = 'lon'
loopCounter = 0
# for each row
for index, row in csvpd.iterrows() :
    loopCounter += 1
    # if the route number is in the dictionary
    if row[RTNUM] in coordDict :
        # extract the pair list
        coordListPair = coordDict[row[RTNUM]]
        # add the new coordinates to the pair list
        coordListPair[LAT].append(row[LAT])
        coordListPair[LON].append(row[LON])
    else :
        # the route number is not in the dictionary
        coordListPair = {}
        # initialize the data structure
        coordListPair[LAT] = []
        coordListPair[LON] = []
        # insert it into the dictionary
        coordDict[row[RTNUM]] = coordListPair
# create frame for output
frameOut = pd.DataFrame(columns=[RTNUM, LAT, LON])
# for each route number, find the average coordinates
for key, value in coordDict.items() :
    latAvg = avgList(value[LAT])
    lonAvg = avgList(value[LON])
    series1 = pd.Series({RTNUM : key, LAT : latAvg, LON : lonAvg})
    frameOut = pd.concat([frameOut, series1.to_frame().T], ignore_index=True)
print("loopCounter",loopCounter,"coordDict",len(coordDict))
print(frameOut.shape)
frameOut.head()
frameOut.to_csv('RouteUnitCoordinates.csv',index=False)

loopCounter 12064 coordDict 1183
(1183, 3)


In [3]:
# compute scores for each run
countscsv = pd.read_csv('../input/cs522-paper/Counts.csv')
RUNID = 'RunID'
CALLINGINDEX='CallingIndex'
# count unique calling index to check
countscsv[CALLINGINDEX].unique()
# create output data struct
runscore = {}
# for each row
for index, row in countscsv.iterrows() :
    # if the run id is in the dictionary
    if row[RUNID] in runscore :
        # get the list of calling index scores
        scoreDict = runscore[row[RUNID]]
        # get the calling index for this row
        callingindex = row[CALLINGINDEX]
        # increment the number of times this calling index has been seen in this run id
        scoreDict[callingindex] += 1
    else :
        # the run id is not in the dictionary
        scoreDict = {}
        # initialize
        scoreDict[1] = 0
        scoreDict[2] = 0
        scoreDict[3] = 0
        # insert
        runscore[row[RUNID]] = scoreDict
# create frame for output
frameOut2 = pd.DataFrame(columns=[RUNID, "1", "2", "3"])
loopCounter = 0
for key, value in runscore.items() :
    loopCounter += 1
    series2 = pd.Series({RUNID : key, "1" : value[1], "2" : value[2], "3" : value[3]})
    frameOut2 = pd.concat([frameOut2,series2.to_frame().T], ignore_index=True)
    if (loopCounter % 1000) == 0 :
        print(loopCounter)
print(frameOut2.shape)
frameOut2.head()
frameOut2.to_csv("RunScores.csv", index=False)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
(21406, 4)


In [4]:
# we need to find the months in which surveys take place, ah ha, some are in december of the previous calendar year
# using the survey date and run number columns an regex in Notepad++, the only surveys which start in the previous calendar year start in December
surveydate='SurveyDate'
runscsv = pd.read_csv('../input/cs522-paper/Runs.csv', parse_dates=[surveydate])
surveydates = runscsv[surveydate]
months = (surveydates.dt.month).drop_duplicates()
print(months)

0        3
1        4
8        1
29       5
31       2
46       6
69       7
106      8
2560    10
4945     9
5845    12
Name: SurveyDate, dtype: int64


In [5]:
# use the scores and run metadata to create series out of the runs, each series belongs to a route and a year, worse, to make a time series, they have to be of uniform length
# using 60/61 time units, we need to convert each run date to the ordinal day, make a new column in the data frame with that
# I've confirmed there are only four unique numbers for RunNumber, so if I just start counting down I'll hit the largest run number and make that one equal to 60
TIME_UNITS = 60
ORDINAL_DATE = 'ORDINAL_DATE'
SURVEY_YEAR = 'SurveyYear'
ROUTE_NUMBER = "RouteNumber"
RUN_NUMBER = 'RunNumber'
runscsv[ORDINAL_DATE] = surveydates.dt.day_of_year
#print(runscsv[ORDINAL_DATE].drop_duplicates()) #check if the operation seems to work as intended

# iterate through each row, building up the dictionary of runs for a given pair of survey year and route number
# each element of the dictionary of runs should be a pair, the run id and the ordinal day of the year, and indexed by the run number
deletedFromLackOfRuns = 0
surveyyear_routenumber = {}
for index, row in runscsv.iterrows() :
    # create the key
    key_syrn = str(str(row[SURVEY_YEAR]) + str(row[ROUTE_NUMBER]))
    if key_syrn not in surveyyear_routenumber :
        surveyyear_routenumber[key_syrn] = dict([('year', row[SURVEY_YEAR]), ('route', row[ROUTE_NUMBER]), ('runs', {}), ('norm', {})])
        #if row[RUN_NUMBER] in surveyyear_routenumber[key_syrn]['runs'] :
            #print ("very unexpected, assert? throw? I dunno", index, key_syrn, row[RUN_NUMBER])
    #else :
        # initialize the dictionary for this 
        #surveyyear_routenumber[key_syrn] = dict([('year', row[SURVEY_YEAR]), ('route', row[ROUTE_NUMBER]), ('runs', {})])
    # record the day of the year for this run
    surveyyear_routenumber[key_syrn]['runs'][row[RUN_NUMBER]] = (row[ORDINAL_DATE], row[RUNID])
print("done1", len(surveyyear_routenumber))
# normalize dates to range 0-60
for key, value in surveyyear_routenumber.items() :
    runs = value['runs']
    #print(runs)
    handleCases = True
    # handle case where we only have one run
    if len(runs) == 1 :
        deletedFromLackOfRuns += 1
        handleCases = False
        del value['runs']
        continue
    # handle the case where we're lacking run number 1
    if 1 not in runs and handleCases and 2 in runs:
        deletedFromLackOfRuns += 1
        handleCases = False
        del value['runs']
        continue
        #runs[1] = runs[2]
        #runs[2] = runs[3]
        #if 4 in runs :
        #    runs[3] = runs[4]
        #    del runs[4]
        #else :
        #    del runs[3]
    # handle missing 1 and 2
    if 1 not in runs and handleCases and 2 not in runs :
        deletedFromLackOfRuns += 1
        handleCases = False
        del value['runs']
        continue
        #runs[1] = runs[3]
        #runs[2] = runs[4]
        #del runs[3]
        #del runs[4]
    # handle missing run number 2
    if 2 not in runs and handleCases:
        deletedFromLackOfRuns += 1
        handleCases = False
        del value['runs']
        continue
        #runs[2] = runs[3]
        #del runs[3]
        #if 4 in runs :
        #    runs[3] = runs[4]
        #    del runs[4]
    # extract the run numbers
    (a,b) = runs[1] # (ordinal, run id)
    (c,d) = runs[2] if 2 in runs else (False, False)
    (e,f) = runs[3] if 3 in runs else (False, False)
    # handle case where the first date is in the previous calendar year, the month of which is only December
    (g,h) = runs[4] if 4 in runs else (False, False)
    if c and a > c :
        if c :
            c = c + 365
        if e :
            e = e + 365
        if g :
            g = g + 365
    # figure out the range of dates, normalize them to start at 0
    runrange = 0
    rangefactor = 1
    if g :
        runrange = g - a
    elif e :
        runrange = e - a
    else :
        if c :
            runrange = c - a
    if runrange > 0 :
        rangefactor = 60.0 / runrange    
    if c :
        c = c - a
    if e :
        e = e - a
    if g :
        g = g - a
    a = 0
    # normalize them to fit between 0 and 60
    if c :
        c = c * rangefactor
    if e :
        e = e * rangefactor
    if g :
        g = g * rangefactor
    # put them back into the dictionary
    runs[1] = (a,b)
    if c :
        runs[2] = (c,d)
    if e :
        runs[3] = (e,f)
    if g :
        runs[4] = (g,h)
print("done2",deletedFromLackOfRuns)
# now we need to put this dictionary in a form we can use later
# create .csv for writing
f = open("consolidated.csv", "wb")
# write csv headers
f.write("SurveyYear,RouteNumber,RunNumber1,OrdinalDate1,RunID1,RunNumber2,OrdinarlDate2,RunID2,RunNumber3,OrdinalDate3,RunID3,RunNumber4,OrdinalDate4,RunID4\n".encode())
# loop through dict
tempcnt = 0
for key, value in surveyyear_routenumber.items() :
    if 'runs' in value :
        stracc = str(value['year']) + ','
        stracc += (str(value['route']) + ',')
        runs = value['runs']
        if 1 in runs :
            (ord, runid) = runs[1]
            stracc += ("1," + str(ord) + "," + str(runid) + ",")
        else :
            stracc += ",,,"
        if 2 in runs :
            (ord, runid) = runs[2]
            stracc += ("2," + str(ord) + "," + str(runid) + ",")
        else :
            stracc += ",,,"
        if 3 in runs :
            (ord, runid) = runs[3]
            stracc += ("3," + str(ord) + "," + str(runid) + ",")
        else :
            stracc += ",,,"
        if 4 in runs :
            (ord, runid) = runs[4]
            stracc += ("4," + str(ord) + "," + str(runid) + "\n")
        else :
            stracc += ",,\n"
        f.write(stracc.encode())
        tempcnt += 1
        if tempcnt < 30 :
            print(stracc)
# finished writing, close file
f.close()

done1 7678
done2 1560
2001,510503,1,0,136,2,34.8,177,3,60.0,197,,,

2001,211101,1,0,137,2,26.129032258064516,179,3,60.0,192,,,

2001,460503,1,0,138,2,28.314606741573034,304,3,60.0,305,,,

2001,460203,1,0,139,2,34.285714285714285,9825,3,60.0,262,,,

2001,460704,1,0,141,2,31.304347826086957,321,3,60.0,9681,,,

2001,460708,1,0,9730,2,29.690721649484537,9731,3,60.0,9732,,,

2001,460505,1,0,143,2,20.666666666666664,309,3,60.0,310,,,

2001,211102,1,0,145,2,29.18918918918919,785,3,60.0,243,,,

2001,420211,1,0,146,2,24.17910447761194,147,3,60.0,189,,,

2001,420311,1,0,153,2,17.00787401574803,148,3,60.0,208,,,

2001,420316,1,0,149,2,17.00787401574803,150,3,60.0,209,,,

2001,420310,1,0,151,2,39.25233644859813,214,3,59.99999999999999,225,,,

2001,420411,1,0,152,2,23.149606299212596,219,3,60.0,218,,,

2001,420312,1,0,154,2,23.243243243243246,155,3,60.0,296,,,

2001,420210,1,0,156,2,20.0,223,3,60.0,224,,,

2001,420110,1,0,157,2,18.52941176470588,220,3,60.0,227,,,

2001,420516,1,0,158,2,31.428571428

In [6]:
# need a function to create scores, except no I don't because the files aren't accessible unless you Save Version and run the whole thing
#def runscorer(scale1, scale2, scale3) :
#    scorecsv = pd.read_csv('RunScores.csv')
#    scorecsv['1'] = scorecsv['1'].apply(lambda x : x * scale1)
#    scorecsv.to_csv('RunScoreTotal.csv', index=False)
#    print(scorecsv.head())
#runscorer(2,1,1)
#print(os.getcwd())
os.listdir(os.getcwd())

['consolidated.csv',
 '__notebook__.ipynb',
 'RunScores.csv',
 'RouteUnitCoordinates.csv']